In [1]:
! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/ensemble ./

In [2]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [3]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [4]:
def ds_from_df(df, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Meme')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [5]:
from tensorflow import keras

alex = keras.models.load_model('ensemble/alexnet_ffwd')
dense = keras.models.load_model('ensemble/densenet_ffwd')
resnet = keras.models.load_model('ensemble/resnet_ffwd')

In [6]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [7]:
import numpy as np
import pickle

des_X, alex_X, res_X, y = pickle.load(open('densenest_embeddings.pkl', 'rb')), pickle.load(open('alexnet_embeddings.pkl', 'rb')), np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [9]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ohe(create_map())
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = pickle.load(open('cls_umberto_embeddings.pkl', 'rb'))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

(1600, 1)
(1600, 1001)
(1600, 4097)
(1600, 2049)
(1600, 1)
(1600, 1002)
(1600, 4098)
(1600, 2050)
(1600, 71)
(1600, 1073)
(1600, 4169)
(1600, 2121)
(1600, 768)
(1600, 1841)
(1600, 4937)
(1600, 2889)


In [10]:
from sklearn.model_selection import train_test_split

res_X_train, res_X_test, res_y_train, res_y_test = train_test_split(res_X, y, test_size=0.2, stratify=y, random_state=42)
des_X_train, des_X_test, des_y_train, des_y_test = train_test_split(des_X, y, test_size=0.2, stratify=y, random_state=42)
alex_X_train, alex_X_test, alex_y_train, alex_y_test = train_test_split(alex_X, y, test_size=0.2, stratify=y, random_state=42)

In [11]:
buffer_size = 10000
batch_size = 64
num_epochs = 10

In [12]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

ds_res = tf.data.Dataset.from_tensor_slices((res_X_test, res_y_test))
ds_res = ds_res.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_res = ds_res.batch(batch_size)

ds_des = tf.data.Dataset.from_tensor_slices((des_X_test, des_y_test))
ds_des = ds_des.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_des = ds_des.batch(batch_size)

ds_alex = tf.data.Dataset.from_tensor_slices((alex_X_test, alex_y_test))
ds_alex = ds_alex.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_alex = ds_alex.batch(batch_size)

In [13]:
# del a, d, r

a = alex.fit(ds_alex,
             epochs=num_epochs,
             batch_size=batch_size,
             use_multiprocessing=True)

d = dense.fit(ds_des,
              epochs=num_epochs,
              batch_size=batch_size,
              use_multiprocessing=True)
              
r = resnet.fit(ds_res,
               epochs=num_epochs,
               batch_size=batch_size,
               use_multiprocessing=True)

Epoch 1/10
5/5 [==============================] - 0s 60ms/step - loss: 0.7099 - binary_accuracy: 0.7344
Epoch 2/10
5/5 [==============================] - 0s 47ms/step - loss: 0.3551 - binary_accuracy: 0.8781
Epoch 3/10
5/5 [==============================] - 0s 46ms/step - loss: 0.2218 - binary_accuracy: 0.9375
Epoch 4/10
5/5 [==============================] - 0s 49ms/step - loss: 0.0991 - binary_accuracy: 0.9656
Epoch 5/10
5/5 [==============================] - 0s 48ms/step - loss: 0.0452 - binary_accuracy: 0.9812
Epoch 6/10
5/5 [==============================] - 0s 47ms/step - loss: 0.0166 - binary_accuracy: 0.9906
Epoch 7/10
5/5 [==============================] - 0s 46ms/step - loss: 0.0181 - binary_accuracy: 0.9937
Epoch 8/10
5/5 [==============================] - 0s 48ms/step - loss: 0.0245 - binary_accuracy: 0.9906
Epoch 9/10
5/5 [==============================] - 0s 46ms/step - loss: 0.0055 - binary_accuracy: 0.9969
Epoch 10/10
5/5 [==============================] - 0s 45ms/step 

In [14]:
alex.save('models/ensemble_eval/alexnet_ffwd')
dense.save('models/ensemble_eval/densenet_ffwd')
resnet.save('models/ensemble_eval/resnet_ffwd')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/ensemble_eval/alexnet_ffwd/assets
INFO:tensorflow:Assets written to: models/ensemble_eval/densenet_ffwd/assets
INFO:tensorflow:Assets written to: models/ensemble_eval/resnet_ffwd/assets


In [15]:
! cp -r models ./drive/My\ Drive/Colab\ Notebooks/